In [42]:
import pandas as pd
import geopandas as gpd

# File paths
csv_file = "../../../_data/archive/ccq nhsd data/output/spatial_join.csv"
shapefile = "../../../_data/archive/ccq nhsd data/input/SA2_2016_Qld.shp"

# read the file, and keep only two fields: SA2_MAIN16 and Name
df = pd.read_csv(csv_file, usecols=['SA2_MAIN16', 'Name'])

# print df head and length
print(df.head())
print(len(df))

# read the shapefile, and create a dict from values from field SA2_MAIN16: AREASQKM16, and print the dict
gdf = gpd.read_file(shapefile)
sa2_dict = dict(zip(gdf.SA2_MAIN16, gdf.AREASQKM16))
print(sa2_dict)

   SA2_MAIN16                                            Name
0   301011001                General practitioner : 0 - 10000
1   301011001  Social and mental support services : 0 - 10000
2   301011001   General community health services : 0 - 10000
3   301011001          Healthy lifestyle services : 0 - 10000
4   301011001                  Aged care services : 0 - 10000
421390
{'301011001': 13.6312, '301011002': 33.1072, '301011003': 11.5987, '301011004': 18.889, '301011005': 2.6658, '301011006': 9.404, '301021007': 11.8548, '301021008': 4.929, '301021009': 46.8997, '301021011': 65.1972, '301021012': 21.6475, '301021013': 13.4057, '301021527': 317.0444, '301031014': 32.7859, '301031015': 4.1989, '301031016': 4.9055, '301031017': 8.4389, '301031018': 8.9263, '301031019': 5.1962, '301031020': 6.1324, '301031021': 12.2743, '302011022': 13.3544, '302011023': 9.4789, '302011024': 4.5457, '302011025': 4.1663, '302011026': 4.3856, '302021027': 6.258, '302021028': 3.3875, '302021029': 3.4032, '

In [43]:
# modify the Name field, and keep only the first part of the string like from General practitioner : 0 - 10000 to General practitioner, and remove the last space
df['Name'] = df['Name'].str.split(':').str[0].str.rstrip()

# print df head
print(df.head())

   SA2_MAIN16                                Name
0   301011001                General practitioner
1   301011001  Social and mental support services
2   301011001   General community health services
3   301011001          Healthy lifestyle services
4   301011001                  Aged care services


In [44]:
# create unique types from Name field, getting rid of nan and values if "Location" is within it, print list and length
types = df['Name'].unique().tolist()
types = [x for x in types if str(x) != 'nan']
types = [x for x in types if "Location" not in x]
print(types)
print(len(types))

['General practitioner', 'Social and mental support services', 'General community health services', 'Healthy lifestyle services', 'Aged care services', 'Other disease management', 'Other', "Women and children's health services", 'Other general hospital and emergency services', 'Breast cancer treatment facilities', 'Medical imaging services', 'Pathology facilities', 'Other cancer services']
13


In [45]:
from tqdm import tqdm

# create a new dataframe with rows corresponding to sa2_dict keys, and columns corresponding to types
# each cell value should be the number of times the type appears in the df for that sa2, if there is no appearance, the cell value should be 0
# if sa2 code does not appear in the df, the row should still be created, with all cell values 0
# print the head of the new dataframe
df2 = pd.DataFrame(index=sa2_dict.keys(), columns=types)
df2 = df2.fillna(0)
print(df2.head())
print(len(df2))

           General practitioner  Social and mental support services  \
301011001                     0                                   0   
301011002                     0                                   0   
301011003                     0                                   0   
301011004                     0                                   0   
301011005                     0                                   0   

           General community health services  Healthy lifestyle services  \
301011001                                  0                           0   
301011002                                  0                           0   
301011003                                  0                           0   
301011004                                  0                           0   
301011005                                  0                           0   

           Aged care services  Other disease management  Other  \
301011001                   0                     

In [46]:
for index, row in tqdm(df.iterrows(), total=len(df), desc="Iterating over df"):
    if str(row['SA2_MAIN16']) not in sa2_dict.keys() or row['Name'] not in types:
        continue
    df2.loc[str(row['SA2_MAIN16']), row['Name']] += 1
print(df2.head())

Iterating over df: 100%|██████████| 421390/421390 [02:11<00:00, 3215.83it/s]

           General practitioner  Social and mental support services  \
301011001                    58                                  71   
301011002                   168                                 304   
301011003                    83                                 120   
301011004                    95                                 123   
301011005                    80                                  94   

           General community health services  Healthy lifestyle services  \
301011001                                 51                          52   
301011002                                182                         199   
301011003                                 70                          78   
301011004                                 77                          87   
301011005                                 64                          73   

           Aged care services  Other disease management  Other  \
301011001                  76                     

In [47]:
# create other more fields with the same number of elements from types, each with string suffiex "density", e.g., "General practitioner density"
# for each row, divide the number of each type by the area of the sa2, and assign the result to the corresponding density field
# print the head of the new dataframe
for type in types:
    df2[type + " density"] = df2[type] / df2.index.map(sa2_dict)
print(df2.head())
print(len(df2))

           General practitioner  Social and mental support services  \
301011001                    58                                  71   
301011002                   168                                 304   
301011003                    83                                 120   
301011004                    95                                 123   
301011005                    80                                  94   

           General community health services  Healthy lifestyle services  \
301011001                                 51                          52   
301011002                                182                         199   
301011003                                 70                          78   
301011004                                 77                          87   
301011005                                 64                          73   

           Aged care services  Other disease management  Other  \
301011001                  76                     

In [48]:
# save the output csv to "../../../_data/archive/ccq nhsd data/output/health_service_count_and_density.csv"
df2.to_csv("../../../_data/archive/ccq nhsd data/output/health_service_count_and_density.csv")

In [49]:
# convert the dataframe to a geodataframe, using the geometry information from the original shapefile through looking up sa2 code
# do not add any other fields from gdf to the output, other than SA2_MAIN16 and geometry
# save the output shapefile to "../../../_data/archive/ccq nhsd data/output/health_service_count_and_density.shp"
gdf2 = gdf[['SA2_MAIN16', 'geometry']].merge(df2, left_on='SA2_MAIN16', right_index=True)
gdf2.to_file("../../../_data/archive/ccq nhsd data/output/health_service_count_and_density.shp")

C:\Users\sasir\AppData\Local\Temp\ipykernel_11664\2133678474.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf2.to_file("../../../_data/archive/ccq nhsd data/output/health_service_count_and_density.shp")
